In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [6]:
import pandas as pd
import plotly.graph_objects as go

def create_rankings(df, years):
    rank_columns = ["rank_{}".format(i) for i in range(len(years))]
    for i, year in enumerate(years):
        df[rank_columns[i]] = df.groupby('year')['new_rating'].rank(ascending=False)
        
    return df, rank_columns

def add_text(fig, beer_name, x, y, text, font_size=12):
    fig.add_trace(go.Scatter(x=[x], y=[y], text=[text], mode="text", showlegend=False,
                             textfont=dict(size=font_size), hoverinfo='text+x+y'))

def add_pie_and_flag(fig, x, y, name, ratio):
    fig.add_trace(go.Pie(labels=[name], values=[ratio], hole=0.4,
                         marker=dict(colors=['rgba(255, 255, 255, 0)']),
                         domain=dict(x=[x-0.15, x+0.15], y=[y-0.15, y+0.15]),
                         hoverinfo='label+percent+name', showlegend=False))

years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]
df = pd.read_csv("./../ada-2023-project-laremontada61/top_10_beers_by_year.csv", index_col=None)
df, rank_columns = create_rankings(df, years)


In [7]:
import plotly.graph_objects as go
fig = {
    "data": [],
    "layout": {},
    "frames": []
}

In [8]:
from IPython.display import display, HTML, Markdown, Image

In [9]:
#mettre les paths 
output_plot_folder = "docs/assets/plot/"
output_img_folder = "imgs/"

## 1.4. Functions definition

In [18]:
# A function is defined to save the plots in .html and .png format
def plot_format (fig, name):
    fig.write_html(output_plot_folder + name + ".html")
    fig.write_image(output_img_folder + name + ".png", engine="kaleido")

    #Show plot image in notebook
    return Image(output_img_folder + name + ".png")

In [28]:
import pandas as pd
import plotly.graph_objects as go
from random import sample
import plotly.graph_objects as go
from random import sample
def top_10_year(data, col):

    fig = {
        "data": [],
        "layout": {},
        "frames": []
    }
    # Supposons que vous ayez déjà défini la variable 'data' et 'top_entries' dans votre code précédent.

    # Initialize frames list and sliders dictionary
    frames = []
    sliders_dict = {"steps": []}
    beer_colors = {}

    for year in range(2000, int(data['year'].max()) + 1):
        # Filter data by year
        snap_data = data[data['year'] == year]

        # Sort by new_rating and get top 10
        snap_data = snap_data.sort_values(col, ascending=False).head(10)

        # Sort by new_rating in ascending order
        snap_data = snap_data.sort_values(col, ascending=True)

        # Generate colors for unique beer names
        for item in snap_data['beer_name'].unique():
            if item not in beer_colors:
                beer_colors[item] = 'rgb({}, {}, {})'.format(*sample(range(256), 3))

        snap_data['color'] = snap_data['beer_name'].map(beer_colors)
        max_rating_value = data[col].max()
        # Create frame
        frame = go.Frame(
            data=[
                go.Bar(
                    x=snap_data[col],
                    marker_color=snap_data['color'],
                    orientation='h',
                    hoverinfo='all',
                    text=snap_data['beer_name'],
                    textposition='outside',
                    width=0.5
                )
            ],
            layout=go.Layout(
                font={'size': 14},
                plot_bgcolor='white',
                xaxis={
                    'showline': True,
                    'visible': True,
                    'range': (0, max_rating_value + 2)
                },
                bargap=0,
                bargroupgap=0,
                yaxis={
                    'showline': True,
                    'visible': True,
                    'tickvals': list(range(0, 11)),
                    'ticktext': list(range(10, 0, -1)),
                    'title_text': 'Top ' + str(10) + ' Beers'
                },
                title=f'Top 10 Beers in {year}',
            ),
            name=str(year),
        )

        # Append frame to list of frames
        frames.append(frame)

        # Define slider step
        slider_step = {
            "args": [
                [str(year)],
                {
                    "frame": {
                        "duration": 300,
                        "redraw": True
                    },
                    "mode": "immediate",
                    "transition": {
                        "duration": 300
                    }
                }
            ],
            "label": str(year),
            "method": "animate"
        }
        # Append to sliders_dict
        sliders_dict["steps"].append(slider_step)

    # Create figure
    fig = go.Figure(
        data=[go.Bar()],
        frames=frames,
    )

    # Update layout with play and pause buttons
    fig.update_layout(
        title='Top 10 Beers in 2000',
        font={'size': 15},
        plot_bgcolor='white',
        xaxis={
            'showline': True,
            'visible': True,
            'title_text': col,
            'range': [0, max_rating_value+2]
        },
        yaxis={
            'showline': False,
            'visible': True,
            'tickvals': list(range(0, 11)),
            'ticktext': list(range(10, 0, -1)),
            'title_text': 'Top 10 Beers'
        },
        height=800,
        width=800,
        margin=dict(
            l=100,
            t=50
        ),
        sliders=[sliders_dict],
    )
    fig.write_html(output_plot_folder + "top_10" + ".html")
    return 

In [26]:
!pip install kaleido


In [29]:
top_10_year(df, 'new_rating')